<!-- # Compute sea level variables -->

The purpose of this notebook is to compute the dynamic sea level and the effective sea level.

In [1]:
import os
import sys
import json
from datetime import datetime
from itertools import product
import numpy as np
import xarray as xr
import dask
from dask_jobqueue import PBSCluster
from dask.distributed import Client

sys.path.append('..')
import utils

~/.conda/envs/causality/lib/python3.11/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
## Globals
with open("../paths.json") as paths_json: 
    PATHS = json.load(paths_json)
with open("../globals.json") as globals_json:
    GLOBALS = json.load(globals_json)
    
FILE  = os.path.join(os.path.abspath('.'), 'compute_sea_level_vars.ipynb')

Functions

In [4]:
def compute_zos(ssh_2, areas):
    # Compute GMSL and zos
    gmsl = ssh_2.weighted(areas).mean(dim=('lat', 'lon'))
    zos = ssh_2 - gmsl
    
    # Convert to meters
    zos = zos / 100
    
    # Units
    zos.name = 'zos'
    zos.attrs['long_name'] = 'dynamic_sea_level'
    zos.attrs['units'] = 'm'
    zos.attrs['history'] = (
        f"Created on {datetime.now()} using {FILE}"
    )
    
    return zos

def compute_effective_sea_level(zos, psl, areas, mask):    
    # Spatial anomaly over ocean
    psl_ocn = xr.where(mask, psl, np.nan)
    p_avg = psl_ocn.weighted(areas).mean(dim=('lat','lon'))
    p_anom = psl_ocn - p_avg
    
    # Inverse barometer contribution
    eta_ib = - p_anom / (GLOBALS['rho_ref'] * GLOBALS['gravity'])
    eta_ib.attrs['units'] = 'm'
    
    # Effective sea level
    ssh = zos + eta_ib
    
    # Add metadata
    ssh.name = 'SSH'
    ssh.attrs['units'] = 'm'
    ssh.attrs['long_name'] = 'effective_sea_level'
    ssh.attrs['description'] = (
        "Effective sea level, including the inverse-barometer "
        "contribution to sea level."
    )
    ssh.attrs['history'] =  (
        f"Created on {datetime.now()} using {FILE}"
    )
    
    return ssh

Dask initialization

In [5]:
cluster = PBSCluster()
client = Client(cluster)

In [6]:
cluster.scale(4)
cluster

Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/abrettin/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.54:37125,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/abrettin/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Main

In [7]:
def main():
    START_TIME = datetime.now()

    for member, init_year in product(GLOBALS['members'], GLOBALS['init_years']):
        print(f"LE-{init_year}.{member}")
        print(datetime.now() - START_TIME)

        # Load SSH_2, PSL, areas, mask
        ssh_2 = utils.data.load_dataset('SSH_2', init_year, member)
        psl = utils.data.load_dataset('PSL', init_year, member)
        areas = utils.data.load_areas()
        mask = utils.data.load_mask()

        # Compute zos and write to file
        print(f"Saving zos to {PATHS['sea_level']}")
        zos = compute_zos(ssh_2, areas)
        zos_ds = xr.Dataset({'zos': zos})

        zos_ds.persist()
        zos_ds.to_zarr(
            os.path.join(
                PATHS['sea_level'],
                f'LE2-{init_year}.{member}.zos.zarr'
            )
        )

        # Compute inverse barometer component to sea level
        print(f"Computing effective sea level for {init_year}.{member}")
        print(datetime.now() - START_TIME)
        with dask.config.set(**{'array.slicing.split_large_chunks': False}):
            ssh = compute_effective_sea_level(zos, psl, areas, mask)
        ssh_ds = xr.Dataset({'SSH': ssh})

        print(f"Saving SSH to {PATHS['sea_level']}")
        ssh_ds = ssh_ds.chunk({'time': 3650})
        ssh_ds.persist()
        ssh_ds.to_zarr(
            os.path.join(
                PATHS['sea_level'],
                f'LE2-{init_year}.{member}.SSH.zarr'
            )
        )

        print(f"Saved effective sea level for {init_year}.{member}")
        print("Restarting client")
        print(datetime.now() - START_TIME)

        client.restart()

    print("PROCESS_COMPLETED")
    print(datetime.now() - START_TIME)
    
    return 0

In [8]:
main()

LE-1251.011
0:00:00.000033


$WORK/conda-envs/s2s/lib/python3.8/site-packages/xarray/core/indexing.py:1379: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


Saving zos to $SCRATCH/sea_level/
Computing effective sea level for 1251.011
0:00:45.906961
Saving SSH to $SCRATCH/sea_level/
Saved effective sea level for 1251.011
Restarting client
0:02:12.164191
LE-1281.011
0:02:15.896345


$WORK/conda-envs/s2s/lib/python3.8/site-packages/xarray/core/indexing.py:1379: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


Saving zos to $SCRATCH/sea_level/
Computing effective sea level for 1281.011
0:02:43.108759
Saving SSH to $SCRATCH/sea_level/
Saved effective sea level for 1281.011
Restarting client
0:04:11.740143
LE-1301.011
0:04:14.968683


...

$WORK/conda-envs/s2s/lib/python3.8/site-packages/xarray/core/indexing.py:1379: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


Saving zos to $SCRATCH/sea_level/
Computing effective sea level for 1281.013
0:15:46.885064
Saving SSH to $SCRATCH/sea_level/
Saved effective sea level for 1281.013
Restarting client
0:17:00.826736
LE-1301.013
0:17:04.251026


$WORK/conda-envs/s2s/lib/python3.8/site-packages/xarray/core/indexing.py:1379: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


Saving zos to $SCRATCH/sea_level/
Computing effective sea level for 1301.013
0:17:41.949497
Saving SSH to $SCRATCH/sea_level/
Saved effective sea level for 1301.013
Restarting client
0:19:16.933393
PROCESS_COMPLETED
0:19:20.154512


0

In [9]:
client.close()
cluster.close()

In [ ]:
ds = xr.open_zarr(os.path.join(PATHS['sea_level'], 'LE2-1251.011.SSH.zarr'))

In [25]:
gmsl

<xarray.DataArray (time: 91615)>
array([0.19414432, 0.17879406, 0.1696119 , ..., 0.03579624, 0.02154988,
       0.01991646])
Coordinates:
  * time     (time) object 1850-01-02 00:00:00 ... 2101-01-01 00:00:00